# Modeling damages as IFC entities

In [ ]:
import ifcopenshell
from ifcopenshell.api import run

import numpy as np

from glob import glob

## Create the model context, site, building and storey

An IFC file consist of an IfcProject, IfcSite, IfcBuilding, IfcStorey. All other entities are organised in this hierarchy e.g., a wall is assigned to a storey and thus also to a building, site and project. IfcOpenShell API provides convenient functions to generate all these entities and assign the relationships. 

In [ ]:
# Create a blank model
model = ifcopenshell.file()

# All projects must have one IFC Project element
project = run("root.create_entity", model, ifc_class="IfcProject", name="My Project")

# Geometry is optional in IFC, but because we want to use geometry in this example, let's define units
# Assigning without arguments defaults to metric units
run("unit.assign_unit", model)

# Let's create a modeling geometry context, so we can store 3D geometry (note: IFC supports 2D too!)
context = run("context.add_context", model, context_type="Model")

# In particular, in this example we want to store the 3D "body" geometry of objects, i.e. the body shape
body = run("context.add_context", model, context_type="Model",
    context_identifier="Body", target_view="MODEL_VIEW", parent=context)

# Create a site, building, and storey. Many hierarchies are possible.
site = run("root.create_entity", model, ifc_class="IfcSite", name="My Site")
building = run("root.create_entity", model, ifc_class="IfcBuilding", name="Building A")
storey = run("root.create_entity", model, ifc_class="IfcBuildingStorey", name="Ground Floor")

# Since the site is our top level location, assign it to the project
# Then place our building on the site, and our storey in the building
run("aggregate.assign_object", model, relating_object=project, product=site)
run("aggregate.assign_object", model, relating_object=site, product=building)
run("aggregate.assign_object", model, relating_object=building, product=storey)


## Create a simple wall

For demonstration purposes we create a simple wall.

In [ ]:
# Let's create a new wall
wall = run("root.create_entity", model, ifc_class="IfcWall")
# Add a new wall-like body geometry, 5 meters long, 3 meters high, and 200mm thick
representation = run("geometry.add_wall_representation", model, context=body, length=5, height=3, thickness=0.2)
# Assign our new body geometry back to our wall
run("geometry.assign_representation", model, product=wall, representation=representation)

# Place our wall in the ground floor
run("spatial.assign_container", model, relating_structure=storey, product=wall)

## Parse damage coordinates

From the image processing the coordinates of a damage will be handed over as a text file. The coordinates are mirrored, so that the origin is in the upper left corner, x positive to the right and y positive down. Coordinates from the image are in 2D, so an empty column with value 0 is added for the y coordinates. The y-coordinates from the image are the z-coordinates in the IFC model. 

In [ ]:
# read from text file
coord_files = glob("img/*.csv")
print(coord_files)

coord_array_list = []
for f in coord_files:
    coord_array = np.genfromtxt(f, dtype=float, delimiter=" ")
    coord_array_3d = np.insert(coord_array, 1, 0., axis=1)

    x_coords = np.vstack((coord_array_3d[:,0]))
    y_coords = np.vstack((coord_array_3d[:,1]))
    z_coords = np.vstack((coord_array_3d[:,2] * -1))

    coords_mirrored = np.hstack((x_coords, y_coords, z_coords))

    coord_array_list.append(coords_mirrored)

print(coord_array_list)
    

## Create damage object

As no IFC class for damages exists, an IfcBuildingElementProxy is created using IfcOpenShell. The coordinates obtained above are used to create an IfcPolyLine as Extrusion Area for an IfcExtrudedAreaSolid.  

In [ ]:
for coord_array in coord_array_list:

    damage = run("root.create_entity", model, ifc_class = "IfcBuildingElementProxy")
    # add damage representation
    # check https://blenderbim.org/docs-python/autoapi/ifcopenshell/api/geometry/index.html

    point = model.createIfcCartesianPoint((1000., -50., 2000.))
    dir1 = model.createIfcDirection((0., 0., 1.))
    dir2 = model.createIfcDirection((1., 0., 0.))
    damage_placement = model.createIfcAxis2placement3d(point, dir1, dir2)


    ifcpts = []
    for pt in coord_array:
        pt_list = list(pt)
        pt_tuple = tuple(float(val) for val in pt_list)
        print(type(pt_tuple[2]))
        ifc_point = model.createIfcCartesianPoint(pt_tuple)
        ifcpts.append(ifc_point)

    polyline = model.createIfcPolyLine(ifcpts)

    ifcclosedprofile = model.createIfcArbitraryClosedProfileDef("AREA", None, polyline)
    ifcdir = model.createIfcDirection((0.0, 1.0, 0.))
    solid = model.createIfcExtrudedAreaSolid(ifcclosedprofile, damage_placement, ifcdir, 100.)

    damage_representation = model.createIfcShapeRepresentation(context, "Body", "SweptSolid", [solid])
    # product_shape = model.createIfcProductDefinitionShape(None, None, [damage_representation])

    # Assign our new body geometry back to our wall
    run("geometry.assign_representation", model, product=damage, representation=damage_representation)

    # Place our wall in the ground floor
    run("spatial.assign_container", model, relating_structure=storey, product=damage)






## Write to file

In [ ]:
# Write out to a file
model.write("model.ifc")